In [1]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go

#### Dash is running on http://127.0.0.1:8050/

In [2]:
#Load data

KNN_neighbours = pd.read_csv(os.getcwd() + '/KNN_neighbours_select.csv')
missing = pd.read_csv(os.getcwd() + '/before_imputing.csv')
imputed = pd.read_csv(os.getcwd() + '/score_imputed.csv')
week = pd.read_csv(os.getcwd() + '/week_prediction.csv')
month = pd.read_csv(os.getcwd() + '/month_prediction.csv')
three_month = pd.read_csv(os.getcwd() + '/3month_prediction.csv')

In [3]:
KNN_neighbours.rename(columns={"vader_score": "Original (with missing data)", "KNN_neighbours2": "KNN: 2 Nearest Neighbours", "KNN_neighbours3": "KNN: 3 Nearest Neighbours", "KNN_neighbours4": "KNN: 4 Nearest Neighbours", "KNN_neighbours5": "KNN: 5 Nearest Neighbours"}, inplace=True)

In [4]:
KNN_neighbours.head()

Date  Original (with missing data)  KNN: 2 Nearest Neighbours  \
0  2009-04-06                           0.0                    0.00000   
1  2009-04-07                           0.0                    0.00000   
2  2009-04-08                           NaN                    0.03095   
3  2009-04-09                           NaN                    0.11990   
4  2009-04-10                           NaN                    0.11990   

   KNN: 3 Nearest Neighbours  KNN: 4 Nearest Neighbours  \
0                   0.000000                   0.000000   
1                   0.000000                   0.000000   
2                   0.020633                   0.021925   
3                   0.079933                   0.064500   
4                   0.086000                   0.064500   

   KNN: 5 Nearest Neighbours  
0                    0.00000  
1                    0.00000  
2                    0.05312  
3                    0.05676  
4                    0.05160

In [5]:
joined = pd.merge(missing,imputed,on='Date', how = 'right')

In [6]:
joined = joined.rename(columns={'vader_score_x': 'Sentiment score (with missing data)', 'vader_score_y': 'Sentiment score'})

In [7]:
joined.head()

Date  Sentiment score (with missing data)  Sentiment score
0  2009-04-06                                  0.0         0.000000
1  2009-04-07                                  0.0         0.000000
2  2009-04-08                                  NaN         0.020633
3  2009-04-09                                  NaN         0.079933
4  2009-04-10                                  NaN         0.086000

In [8]:
forecast = pd.merge(pd.merge(pd.merge(imputed, week, on='Date', how = 'outer'), month, on='Date', how = 'outer'), three_month, on='Date', how = 'outer')

In [9]:
forecast

Date  vader_score  Predictions_x  Predictions_y  Predictions
0    2009-04-06     0.000000            NaN            NaN          NaN
1    2009-04-07     0.000000            NaN            NaN          NaN
2    2009-04-08     0.020633            NaN            NaN          NaN
3    2009-04-09     0.079933            NaN            NaN          NaN
4    2009-04-10     0.086000            NaN            NaN          NaN
..          ...          ...            ...            ...          ...
169  2009-09-22          NaN            NaN            NaN     0.001450
170  2009-09-23          NaN            NaN            NaN     0.001183
171  2009-09-24          NaN            NaN            NaN     0.001420
172  2009-09-25          NaN            NaN            NaN     0.001121
173  2009-09-26          NaN            NaN            NaN     0.001313

[174 rows x 5 columns]

In [10]:
forecast = forecast.rename(columns={'vader_score': 'Measured sentiment score', 'Predictions_x': '1 week forecast', 'Predictions_y': '1 month forecast', 'Predictions': '3 month forecast'})

In [11]:
forecast.head()

Date  Measured sentiment score  1 week forecast  1 month forecast  \
0  2009-04-06                  0.000000              NaN               NaN   
1  2009-04-07                  0.000000              NaN               NaN   
2  2009-04-08                  0.020633              NaN               NaN   
3  2009-04-09                  0.079933              NaN               NaN   
4  2009-04-10                  0.086000              NaN               NaN   

   3 month forecast  
0               NaN  
1               NaN  
2               NaN  
3               NaN  
4               NaN

In [12]:
forecast["1 week forecast"] = forecast["1 week forecast"].fillna(forecast["Measured sentiment score"])

In [13]:
forecast["1 month forecast"] = forecast["1 month forecast"].fillna(forecast["Measured sentiment score"])

In [14]:
forecast["3 month forecast"] = forecast["3 month forecast"].fillna(forecast["Measured sentiment score"])

In [15]:
forecast.head()

Date  Measured sentiment score  1 week forecast  1 month forecast  \
0  2009-04-06                  0.000000         0.000000          0.000000   
1  2009-04-07                  0.000000         0.000000          0.000000   
2  2009-04-08                  0.020633         0.020633          0.020633   
3  2009-04-09                  0.079933         0.079933          0.079933   
4  2009-04-10                  0.086000         0.086000          0.086000   

   3 month forecast  
0          0.000000  
1          0.000000  
2          0.020633  
3          0.079933  
4          0.086000

In [16]:
forecast.tail()

Date  Measured sentiment score  1 week forecast  1 month forecast  \
169  2009-09-22                       NaN              NaN               NaN   
170  2009-09-23                       NaN              NaN               NaN   
171  2009-09-24                       NaN              NaN               NaN   
172  2009-09-25                       NaN              NaN               NaN   
173  2009-09-26                       NaN              NaN               NaN   

     3 month forecast  
169          0.001450  
170          0.001183  
171          0.001420  
172          0.001121  
173          0.001313

In [17]:
# Initialize the app
app = Dash(__name__)

# App layout
app.layout = html.Div([
        html.Div(children='KNN selector', style={"font-weight": "bold"}),
        html.Hr(),
        dcc.RadioItems(options=['Original (with missing data)', 'KNN: 2 Nearest Neighbours', 'KNN: 3 Nearest Neighbours', 'KNN: 4 Nearest Neighbours', 'KNN: 5 Nearest Neighbours'], value='Original (with missing data)', id='KNN_selector'),
        dcc.Graph(figure={}, id='KNN'),

        html.Div(children='After imputation', style={"font-weight": "bold"}),
        html.Hr(),
        dcc.RadioItems(options=['Sentiment score (with missing data)', 'Sentiment score'], value='Sentiment score (with missing data)', id='missing_and_imputed'),
        dcc.Graph(figure={}, id='imputation_complete'),
    
        html.Div(children='Forecast', style={"font-weight": "bold"}),
        html.Hr(),
        dcc.RadioItems(options=['Measured sentiment score', '1 week forecast', '1 month forecast', '3 month forecast'], value='Measured sentiment score', id='forecast'),
        dcc.Graph(figure={}, id='forecasted')
])

# Add controls to build the interacti3n
@callback(
    Output(component_id='KNN', component_property='figure'),
    Input(component_id='KNN_selector', component_property='value')
)
def update_graph(col_chosen):
    fig = px.scatter(KNN_neighbours, x='Date', y=col_chosen, title="Selecting appropriate nearest neighbours for KNN")
    return fig

# Add controls to build the interaction
@callback(
    Output(component_id='imputation_complete', component_property='figure'),
    Input(component_id='missing_and_imputed', component_property='value')
)
def update_graph(col_chosen):
    fig = px.scatter(joined, x='Date', y=col_chosen, title="Imputing missing values")
    return fig

# Add controls to build the interaction
@callback(
    Output(component_id='forecasted', component_property='figure'),
    Input(component_id='forecast', component_property='value')
)
def update_graph(col_chosen):
    fig = px.scatter(forecast, x='Date', y='Measured sentiment score', color_discrete_sequence=['red']),
    fig = px.scatter(forecast, x='Date', y=col_chosen, title="Forecasting sentiment score")
    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=False)